In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
import csv
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, Flatten
from keras.callbacks import EarlyStopping
import numpy as np

In [ ]:
def data():
  x_raw_name=[]
  y_label=[]
  images = []

  image_path = '/content/covid-chestxray-dataset/images'
  image_names=os.listdir(image_path)
  with open('/content/covid-chestxray-dataset/metadata.csv') as csv_file:
      csv_reader = csv.reader(csv_file, delimiter=',')
      for row in csv_reader:
          if row[23] in image_names:
              if row[5]=='':
                  y_label.append(False)
              if row[5]=='Y':
                  y_label.append(True)
              if row[5]=='Y' or row[5]=='':
                  x_raw_name.append(row[23])
                  img = cv2.imread(image_path + '/' + x_raw_name[-1],0)
                  img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
                  images.append(img)
  
  image_path = '/content/Figure1-COVID-chestxray-dataset/images'
  image_names=os.listdir(image_path)
  with open('/content/Figure1-COVID-chestxray-dataset/metadata.csv', encoding='latin1') as csv_file:
      csv_reader = csv.reader(csv_file, delimiter=',')
      for row in csv_reader:
          if (row[0]+'.jpg') in image_names:
              if row[4]=='COVID-19':
                  y_label.append(True)
              else:
                  y_label.append(False)
              x_raw_name.append(row[0]+'.jpg')
              img = cv2.imread(image_path + '/' + x_raw_name[-1],0)
              img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
              images.append(img)

  image_path = '/content/Actualmed-COVID-chestxray-dataset/images'
  image_names=os.listdir(image_path)
  with open('/content/Actualmed-COVID-chestxray-dataset/metadata.csv') as csv_file:
      csv_reader = csv.reader(csv_file, delimiter=',')
      for row in csv_reader:
          if row[10] in image_names:
              if row[4]=='COVID-19':
                  y_label.append(True)
              else:
                  y_label.append(False)
              x_raw_name.append(row[10])
              img = cv2.imread(image_path + '/' + x_raw_name[-1],0)
              img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
              images.append(img)
  
  images = np.asarray(images)
  y_label = np.asarray(y_label)

  normalized = images.astype(float)
  for i in range(len(normalized)):
    normalized[i] = normalized[i]/255

  triplechanel=np.zeros([len(normalized[:,0,0]),len(normalized[0,:,0]),len(normalized[0,0,:]),3])
  triplechanel[:,:,:,0], triplechanel[:,:,:,1], triplechanel[:,:,:,2] = normalized, normalized, normalized
  x_train, x_2, y_train, y_2 = train_test_split(triplechanel, y_label, test_size=0.3, shuffle=True)
  x_valid, x_test, y_valid, y_test = train_test_split(x_2, y_2, test_size=0.33, shuffle=True)

  return x_train, y_train, x_test, y_test

In [ ]:
def create_model():
  baseModel = tf.keras.applications.VGG19(weights="imagenet", include_top=False, input_tensor=tf.keras.layers.Input(shape=(224, 224, 3)))
  output = baseModel.output

  act = {{choice(['relu', 'leakyrelu'])}}
  if act == 'relu':
      activation = keras.layers.ReLU()
  elif act == 'leakyrelu':
      activation = keras.layers.LeakyReLU()

  pooling = {{choice([True, False])}}
  if pooling:
      output = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))(output)
  
  output = tf.keras.layers.Flatten(name="flatten")(output)

  layer1 = {{choice([256, 512, 1024])}}
  output = tf.keras.layers.Dense(layer1, activation=activation)(output)

  output = tf.keras.layers.Dropout(0.5)(output)

  layer2 = {{choice([256, 512, 1024])}}
  output = tf.keras.layers.Dense(layer2, activation=activation)(output)

  output = tf.keras.layers.Dropout(0.5)(output)

  output = tf.keras.layers.Dense(1, activation="sigmoid")(output)

  model = tf.keras.Model(inputs=baseModel.input, outputs=output)
  for layer in baseModel.layers:
    layer.trainable = False
  
  EPOCHS = 40

  batchsize = {{choice([16, 32, 64])}}

  optim = {{choice(['adam', 'sgd'])}}

  model.compile(loss="binary_crossentropy", optimizer=optim, metrics=["accuracy"])

  callbacks = [EarlyStopping(monitor='val_accuracy', patience=3, verbose=0)]

  STEP_TRAIN = len(x_train) // batchsize
  STEP_VALID = len(x_valid) // batchsize
  
  result = model.fit(x_train, y_train, batch_size=batchsize, epochs=EPOCHS, verbose=1,
                         validation_data=(x_valid, y_valid), shuffle = True, steps_per_epoch=STEP_TRAIN, validation_steps=STEP_VALID,callbacks=callbacks)
  
  best_val_acc = np.amax(result.history['val_accuracy'])

  with open('covid-hyperas-log.csv', 'a') as csv_file:
      csv_file.write(str(act) + ',')
      csv_file.write(str(pooling) + ',')
      csv_file.write(str(layer1) + ',')
      csv_file.write(str(layer2) + ',')
      csv_file.write(str(batchsize) + ',')
      csv_file.write(str(optim) + ',')
      csv_file.write(str(best_val_acc) + '\n')

  return {'loss': -best_val_acc, 'status': STATUS_OK, 'model': model}